In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from random import  uniform
from itertools import combinations

In [3]:
num_trials = 100
num_points = 1000
K = 4


In [4]:
points = np.random.random((num_points,2))

In [5]:

if len(points)<10:
    combinations_list = list(combinations(points, K))

    scores = []
    for combo in combinations_list:
        combo = np.array(combo)
        matching_score = 1-np.prod(combo[:,0])
        engagement_score =  np.sum(combo[:,1])
        scores.append(matching_score + engagement_score)
    np.max(scores)

In [6]:
epsilon = .01 
dp = np.zeros((K+1,num_points+1,round(1/epsilon)+1))

In [7]:
for k in range(dp.shape[0]):
    for n in range(dp.shape[1]):
        for e in range(dp.shape[2]):
            if k == 0 or n == 0:
                dp[k][n][e] = 1-e*epsilon
            else: 
                baseline = dp[k][n-1][e]
                other_option = dp[k-1][n-1][round(e*points[n-1][0])] + points[n-1][1]
                dp[k][n][e] = max(baseline,other_option)

In [8]:
dp[-1][-1][-1]

4.995494652146921

In [14]:
def to_prob(x):
    x = np.maximum(x,0)
    return (100-x)/100

num_times = 1000

mu_values = np.array([0.0 for i in range(num_points)])
max_vals = np.array([0.0 for i in range(num_points)])
num_pulls = np.array([0.0 for i in range(num_points)])
observed_vals = np.array([0.0 for i in range(num_points)])

upper_bound_match = np.prod(sorted(points[:,0])[-K-1:])
upper_bound_engagement = np.sum(sorted(points[:,1])[-K-1:])

for i in range(len(mu_values)):
    max_val = 1-points[i][0]*upper_bound_match+points[i][1]+upper_bound_engagement
    max_vals[i] = max_val 
    mu_values[i] = max_val 

for i in range(num_times):
    current_cohort = np.argsort(mu_values)[-k:][::-1]
    real_val = 1-np.prod(points[current_cohort,0]) + np.sum(points[current_cohort,1])
    observed_vals[current_cohort] = np.maximum(observed_vals[current_cohort],real_val)
    num_pulls[current_cohort] += 1

    probs = to_prob(num_pulls[current_cohort])
    mu_values[current_cohort] = probs * max_vals[current_cohort] + (1-probs)*observed_vals[current_cohort]
cohort_vals = []
for i in range(num_times):
    rand_points = points[np.random.choice(points.shape[0], k, replace=False)]
    real_val = 1-np.prod(rand_points[:,0]) + np.sum(rand_points[:,1])
    cohort_vals.append(real_val)
np.max(observed_vals), np.max(cohort_vals)

(4.958504375312464, 4.462580151919658)

In [21]:
greedy_vals = [(1-i[0]+i[1]) for i in points]
max_greedy = np.argsort(greedy_vals)[-k:][::-1]
true_points = points[max_greedy]
score = 1-np.prod(true_points[:,0])+np.sum(true_points[:,1])
score

4.8985039670757216